In [115]:
from models import unetnc
import numpy as np
import pickle as pkl
import cv2
import torch
import cv2

DATA_PATH = 'Dataset Preview/Inv3D preview/data/'
# warped WC sind die 3D Koordinaten nach dem ersten Modell bzw. dewarpnet -> Ground Truth

In [116]:
trainimage_gt = np.load(DATA_PATH + 'train/06/warped_WC.npz')

In [117]:
img_t = trainimage_gt['warped_WC']
#img_t = torch.from_numpy(img).float()

In [118]:
#normalize label
lbl = img_t
msk=((lbl[:,:,0]!=0)&(lbl[:,:,1]!=0)&(lbl[:,:,2]!=0)).astype(np.uint8)*255
xmx, xmn, ymx, ymn,zmx, zmn= 1.2539363, -1.2442188, 1.2396319, -1.2289206, 0.6436657, -0.67492497   # calculate from all the wcs
lbl[:,:,0]= (lbl[:,:,0]-zmn)/(zmx-zmn)
lbl[:,:,1]= (lbl[:,:,1]-ymn)/(ymx-ymn)
lbl[:,:,2]= (lbl[:,:,2]-xmn)/(xmx-xmn)
lbl=cv2.bitwise_and(lbl,lbl,mask=msk)
#lbl = cv2.resize(lbl, self.img_size, interpolation=cv2.INTER_NEAREST)
lbl = lbl.transpose(2, 0, 1)   # NHWC -> NCHW
lbl = np.array(lbl, dtype=np.float64)
lbl

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [79]:
type(trainimage_gt['warped_WC'])

numpy.ndarray

In [82]:
np.load(DATA_PATH + 'train/06/warped_document.png')

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [87]:
from numpy import asarray
from PIL import Image
image = Image.open(DATA_PATH + 'train/01/warped_document.png')
# convert image to numpy array
img = asarray(image)
#img = m.imresize(img, self.img_size) # uint8 with RGB mode
if img.shape[-1] == 4:
    img=img[:,:,:3]   # Discard the alpha channel  
img = img[:, :, ::-1] # RGB -> BGR
# plt.imshow(img)
# plt.show()
img = img.astype(float) / 255.0
img = img.transpose(2, 0, 1) # NHWC -> NCHW


In [89]:
img.shape

(3, 256, 256)

In [86]:
image

array([[[ 29,  26,  24, 255],
        [ 30,  27,  25, 255],
        [ 29,  26,  24, 255],
        ...,
        [ 30,  26,  23, 255],
        [ 31,  27,  24, 255],
        [ 31,  28,  25, 255]],

       [[ 28,  26,  24, 255],
        [ 30,  27,  25, 255],
        [ 29,  27,  24, 255],
        ...,
        [ 29,  26,  23, 255],
        [ 31,  27,  24, 255],
        [ 31,  27,  24, 255]],

       [[ 28,  25,  23, 255],
        [ 29,  26,  24, 255],
        [ 29,  27,  24, 255],
        ...,
        [ 29,  26,  23, 255],
        [ 30,  26,  23, 255],
        [ 29,  26,  23, 255]],

       ...,

       [[ 22,  20,  18, 255],
        [ 23,  20,  18, 255],
        [ 23,  20,  18, 255],
        ...,
        [ 28,  26,  24, 255],
        [ 28,  27,  24, 255],
        [ 29,  27,  25, 255]],

       [[ 23,  20,  18, 255],
        [ 23,  20,  18, 255],
        [ 23,  20,  18, 255],
        ...,
        [ 28,  26,  24, 255],
        [ 28,  26,  24, 255],
        [ 28,  27,  25, 255]],

       [[ 23

In [35]:
DATA_PATH = 'Dataset Preview/Inv3D preview/data/'

from custom_dataset import CustomImageDataset_wc
#train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=True)
train_dataset = CustomImageDataset_wc(data_dir=DATA_PATH+'train/', transform=False)

from models import unetnc
model = unetnc.UnetGenerator(input_nc=3, output_nc=6, num_downs=3)
#model.to('cuda')
from torch.autograd import Variable

In [36]:
from torch.utils import data
#data_loader = get_loader('doc3dwc')
#data_path = args.data_path
#t_loader = data_loader(data_path, is_transform=True, img_size=(args.img_rows, args.img_cols), augmentations=True)
#v_loader = data_loader(data_path, is_transform=True, split='val', img_size=(args.img_rows, args.img_cols))

#n_classes = t_loader.n_classes
trainloader = data.DataLoader(train_dataset, batch_size=1, num_workers=8, shuffle=True)
#valloader = data.DataLoader(v_loader, batch_size=args.batch_size, num_workers=8)

In [76]:
for item in trainloader:
    #print(item)
    pass
img = Variable(item['image'])#.to('cuda'))  #images = Variable(images.cuda())
img.size()

torch.Size([1, 3, 256, 256])

In [ ]:
from torch.autograd import Variable
for i, (images, labels) in enumerate(trainloader):
    print(i)
    print(images)
    print(labels)
    #images = Variable(images.cuda())
    #labels = Variable(labels.cuda())

    #optimizer.zero_grad()
    outputs = model(images)
    '''pred=htan(outputs)
    g_loss=gloss(pred, labels)
    l1loss = loss_fn(pred, labels)
    loss=l1loss#+(0.2*g_loss)
    avg_l1loss+=float(l1loss)
    avg_gloss+=float(g_loss)
    avg_loss+=float(loss)
    train_mse+=float(MSE(pred, labels).item())

    loss.backward()
    optimizer.step()
    global_step+=1'''
    print('lakshooOOOO')
    print(outputs)